In [2]:
import pandas as pd
from datetime import datetime

import great_expectations as gx
from great_expectations.core.expectation_configuration import ExpectationConfiguration

from ydata_profiling import ProfileReport
from ydata_profiling.config import Settings
from ydata_profiling.model import BaseDescription, expectation_algorithms
from ydata_profiling.model.handler import Handler
from ydata_profiling.utils.dataframe import slugify
from ydata_profiling.expectations_report import ExpectationsReport
from great_expectations.checkpoint import SimpleCheckpoint

from typing import Any, Optional

import pandas as pd
from visions import VisionsTypeset
import re
import os

In [86]:
full_path = os.getcwd()
df = pd.read_csv(full_path.partition('notebooks')[0] + 'data/04_feature/customer_features.csv')
#customers = pd.read_csv(full_path.partition('notebooks')[0] + 'data/02_intermediate/customers_validated.csv')
#df = pd.merge(customers, features, left_on='NewId', right_on='CustomerId', how='right').drop(columns='NewId')

In [87]:
df.columns

Index(['CustomerId', 'CreditType', 'CreditAmount', 'Duration_Months',
       'NumberOfInstallmentsToPay', 'PaymentFrequency', 'HasDefault',
       'run_date', 'Avg_Monthly_Income', 'Income_Stability',
       'Avg_Monthly_expenses', 'Expenses_Stability', 'Avg_Monthly_Funds',
       'Funds_Stability', 'Previous_Loan_Count', 'Previous_Loans_Avg_Amount',
       'Previous_Loans_Std', 'Previous_Loan_Defaults', 'Active_Loans_Count',
       'Active_Loan_Amount_Total', 'SegGroup', 'AMLRiskRating',
       'YrNetMonthlyIn', 'Age', 'Is_Employed', 'Is_Married'],
      dtype='object')

In [69]:
l = []
for c in ['customer_since', 'date_of_birth', 'birth_in_corp_date', 'legal_iss_date', 'legal_exp_date']:
    if c in df.columns:
        l.append(c)

l

[]

In [88]:
for c in ['CustomerId',
 'Duration_Months',
 'NumberOfInstallmentsToPay',
 'HasDefault',
 'run_date',
 'Previous_Loan_Count',
 'Previous_Loan_Defaults',
 'Active_Loans_Count',
 'Age',
 'Is_Employed',
 'Is_Married']:
#['NoOfDependents', 'SegmentId', 'IndustryId', 'LegalDocName1Id', 'CustomerId', 'Duration_Months', 'NumberOfInstallmentsToPay', 'HasDefault',
#    'run_date', 'Previous_Loan_Count',  'Previous_Loan_Defaults', 'Active_Loans_Count', 'Age', 'Is_Employed', 'Is_Married']:
    # Convert to Int64 (nullable int)
    df[c] = df[c].fillna(-1)
    df[c] = df[c].astype('int')

df.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in df.columns]
df.columns = df.columns.str.replace('.', '', regex=False)
df.columns = df.columns.str.replace('__', '_', regex=False)

target = ['has_default']
numerical_features = df.select_dtypes('number').drop(columns='has_default').columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
# categorical_features.remove('legal_exp_date')
# datetime_features = ['customer_since', 'date_of_birth', 'birth_in_corp_date', 'legal_iss_date', 'legal_exp_date'] #df.select_dtypes(include=['datetime']).columns.tolist()

for c in categorical_features:
    df[c] = df[c].apply(lambda x: x if pd.notnull(x) else None)


In [27]:
df.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in df.columns]
df.columns = df.columns.str.replace('.', '', regex=False)
df.columns = df.columns.str.replace('__', '_', regex=False)

In [79]:
df[['customer_since', 'date_of_birth', 'birth_in_corp_date', 'legal_iss_date', 'legal_exp_date']].describe()

,customer_since,date_of_birth,birth_in_corp_date,legal_iss_date,legal_exp_date
count,3554,3489,67,3554,3550
unique,367,2818,51,1317,1672
top,2024-03-22,1985-01-01,2008-12-15,20230116,20240524
freq,93,9,9,15,14


In [81]:
pd.to_datetime(df.date_of_birth).sort_values()

56     1952-05-20
19     1954-07-11
81     1955-07-11
932    1957-05-08
103    1957-07-20
          ...    
1947          NaT
1948          NaT
3551          NaT
3552          NaT
3553          NaT
Name: date_of_birth, Length: 3554, dtype: datetime64[ns]

In [75]:
for i in ['placebrth', 'ocupation_desc', 'town_country', 'district', 'legal_iss_auth']:
    print(df[i].nunique())

399
126
471
487
199


In [72]:
df.select_dtypes('int64').describe()

,no_of_dependents,segment_id,industry_id,legal_doc_name1_id,customer_id,duration_months,number_of_installments_to_pay,has_default,run_date,previous_loan_count,previous_loan_defaults,active_loans_count
count,3554.000000,3554.000000,3554.000000,3554.000000,3554.000000,3554.000000,3554.000000,3554.000000,3.554000e+03,3554.000000,3554.000000,3554.000000
mean,-0.990715,1.051491,81627.298818,1.537704,58522.869724,54.293191,53.897580,0.020259,2.024025e+07,0.136466,0.000563,0.075689
std,0.171657,0.354131,15212.775499,3.412344,34904.137427,13.247631,17.135236,0.140904,8.242233e+01,1.027734,0.023719,0.814767
min,-1.000000,1.000000,-1.000000,1.000000,221.000000,11.000000,0.000000,0.000000,2.024013e+07,0.000000,0.000000,0.000000
25%,-1.000000,1.000000,85102.000000,1.000000,27635.500000,60.000000,61.000000,0.000000,2.024013e+07,0.000000,0.000000,0.000000
50%,-1.000000,1.000000,85103.000000,1.000000,57748.500000,60.000000,61.000000,0.000000,2.024023e+07,0.000000,0.000000,0.000000
75%,-1.000000,1.000000,85222.000000,1.000000,89035.500000,60.000000,61.000000,0.000000,2.024033e+07,0.000000,0.000000,0.000000
max,3.000000,11.000000,97000.000000,33.000000,122189.000000,65.000000,66.000000,1.000000,2.024033e+07,29.000000,1.000000,29.000000


In [71]:
df.select_dtypes('float64').describe()

,yr_net_monthly_in,credit_amount,avg_monthly_income,income_stability,avg_monthly_expenses,expenses_stability,avg_monthly_funds,funds_stability,previous_loans_avg_amount,previous_loans_std,active_loan_amount_total
count,3.270000e+02,3.554000e+03,3.554000e+03,3.554000e+03,3.554000e+03,3.554000e+03,3.554000e+03,3.554000e+03,3.554000e+03,3.554000e+03,3.554000e+03
mean,7.369392e+04,3.212828e+05,6.650826e+05,2.984700e+05,6.442183e+05,2.692665e+05,2.627123e+05,6.608653e+04,2.972129e+05,1.247665e+05,3.224817e+05
std,2.077345e+05,5.322488e+06,8.800536e+06,3.201148e+06,8.737515e+06,2.924779e+06,3.745199e+06,1.064594e+06,1.027620e+07,2.434551e+06,6.621845e+06
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.500000e+04,5.257500e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.500000e+04,9.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.912708e+04,1.119931e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.127578e+06,2.573930e+08,3.290531e+08,8.288577e+07,3.304162e+08,6.858092e+07,1.173394e+08,3.642334e+07,6.000000e+08,1.059246e+08,2.814656e+08


In [151]:
for c in ['customer_since', 'date_of_birth', 'birth_in_corp_date']:
    df[c] = pd.to_datetime(df[c], format = "%Y-%m-%d", errors="coerce")
    df[c] = df[c].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None
    )
    #df[c] = df[c].astype("object").where(df[c].notna(), None)


def safe_parse(val):
    try:
        if pd.isna(val):
            return None
        val_str = str(int(val))  # Convert float like 20250101.0 to '20250101'
        return datetime.strptime(val_str, "%Y%m%d").strftime('%Y-%m-%d')
    except ValueError:
        return None  # Or return None

for c in ['legal_iss_date', 'legal_exp_date']:
    df[c] = df[c].str.split('ý').str[-1]
    df[c] = df[c].apply(safe_parse)

#df['legal_exp_date'] = df['legal_exp_date'].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None)

for c in ['no_of_dependents', 'segment_id', 'industry_id', 'legal_doc_name1_id']:
    # Convert to Int64 (nullable int)
    df[c] = df[c].fillna(-1)
    df[c] = df[c].astype('int')

## Split the data in one part as your reference dataset and the other as your analysis dataset

In [155]:
# df_ref = df[df.customer_since.dt.year != 2025]
df_ref = df[~df['customer_since'].str.startswith('2025')]
df_ref

,new_id,customer_since,customer_status,date_of_birth,employment_status,gender,marital_status,placebrth,cust_type,nationality,...,habliter,province,district,legal_doc_name1_id,legal_doc_name1_id_description,legal_iss_date,legal_exp_date,legal_iss_auth,a_m_l_risk_rating,yr_net_monthly_in
0,1,2012-05-23,Private Client - Standard,None,NaN,FEMALE,NaN,NaN,NaN,Portugal,...,NaN,NaN,NaN,-1,NaN,None,None,NaN,Elevado,NaN
1,2,2022-11-30,Private Client - Standard,None,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,-1,NaN,None,None,NaN,Medio,NaN
2,3,2022-11-30,Private Client - Standard,None,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,-1,NaN,None,None,NaN,Medio,NaN
3,4,2022-11-30,Private Client - Standard,None,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,-1,NaN,None,None,NaN,Elevado,NaN
4,5,2013-11-14,Private Client - Standard,2012-07-01,MB,MALE,OTHER,NACALA PORTO,NaN,Mozambique,...,NaN,NAMPULA,NACALA-PORTO,36,Licenca/Alvara,2012-11-13,2050-11-13,NAMPULA,Medio,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122207,122208,2016-11-29,Private Client - Standard,1973-01-16,EMPLOYED,MALE,DIVORCED,ISRAEL,RETAIL,Israel,...,Curso Superior,CABO DELGADO,MAHATE,2,DIRE,2025-05-20,2028-05-19,MIGRACAOýMAIRRO MAHATEýREEF INVESTIMENTOS,Baixo,346500.0
122208,122209,2021-07-23,Private Client - Standard,1978-01-10,EMPLOYED,MALE,SINGLE,ZIMBABWE,NaN,Zimbabwe,...,NaN,MAPUTO,MATOLA 'A',3,Passaporte,2017-06-14,2027-06-13,REPUBLIC OF ZIMBABWE,Medio,3000.0
122209,122210,2017-12-04,Private Client - Standard,1964-01-16,MB,MALE,DIVORCED,HLATHIKHULU,RETAIL,South Africa,...,NaN,NaN,NaN,3,Passaporte,2016-10-13,2026-10-12,NaN,Baixo,NaN
122210,122211,2017-12-04,Corporate - Small,None,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,BAIRRO DE SOMMERSHIELD,24,Certidao da Conservatoria de Regist,2017-12-06,2099-12-06,CONSERVATORIAýDIRECCAO DE CULTURA,Baixo,NaN


In [156]:
# df_ana = df[df.customer_since.dt.year == 2025]
df_ana = df[df['customer_since'].str.startswith('2025')]
df_ana

,new_id,customer_since,customer_status,date_of_birth,employment_status,gender,marital_status,placebrth,cust_type,nationality,...,habliter,province,district,legal_doc_name1_id,legal_doc_name1_id_description,legal_iss_date,legal_exp_date,legal_iss_auth,a_m_l_risk_rating,yr_net_monthly_in
147,148,2025-02-22,Private Client - Standard,None,NaN,NaN,NaN,NaN,NaN,Mozambique,...,NaN,SOFALA,CHAMITE,24,Certidao da Conservatoria de Regist,2025-02-18,2049-02-18,CONSERVATORIA DAS ENTIDADES LEGAISýBAU BEIRA,Baixo,NaN
150,151,2025-02-27,Corporate - Medium,None,NaN,NaN,NaN,NaN,PROSPECT,Mozambique,...,NaN,CIDADE DE MAPUTO,MALANGA,29,Certidao de Registo / Ministerio da,2025-02-13,2050-02-13,CONSERVATORIA ENTIDADES LEGAISýBAU,Elevado,NaN
175,176,2025-05-08,Private Client - Standard,None,NaN,NaN,NaN,NaN,CORPORATE,Mozambique,...,NaN,CABO DELGADO,PEMBA CIDADE,28,Licenca / Alvara para o exercicio d,2025-04-25,2049-04-25,BAUýCONSERVATORIA DAS ENTIDADES LEGAIS,Elevado,NaN
181,182,2025-02-25,Corporate - Small,None,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,MALHANGALAENE 'A',24,Certidao da Conservatoria de Regist,2025-02-12,2049-02-12,MINISTERIO DA JUSTICAýBAU,Medio,NaN
205,206,2025-03-29,Private Client - Standard,None,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,CHAMANCULO 'A',24,Certidao da Conservatoria de Regist,2023-07-13,2099-12-31,MINISTERIO DA JUSTICAýBALCAO ATENDIMENTO UNICO,Elevado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122145,122146,2025-05-16,Private Client - Standard,1981-11-11,NaN,FEMALE,OTHER,PORTUGAL,NaN,Mozambique,...,NaN,NaN,NaN,3,Passaporte,2020-10-20,2025-10-20,NaN,Elevado,NaN
122166,122167,2025-04-29,Private Client - Standard,1986-02-24,NaN,MALE,SINGLE,BEIRA,RETAIL,Mozambique,...,Bacharelato,SOFALA,BEIRA CIDADE,1,BI,2023-08-28,2028-08-27,DIC BEIRA,Elevado,NaN
122191,122192,2025-05-27,Private Client - Standard,1980-07-07,NaN,FEMALE,SINGLE,INHASSUNGE,RETAIL,Mozambique,...,Bacharelato,ZAMBEZIA,MOPEIA,1,BI,2024-07-05,2034-07-04,DIC QUELIMANE,Elevado,NaN
122197,122198,2025-03-21,Private Client - Standard,2024-08-10,NaN,FEMALE,SINGLE,MAPUTO,NaN,Mozambique,...,Ate12 ano,CIDADE DE MAPUTO,ALTO MAE 'B',1,BI,2025-02-28,2030-02-27,DNIC - MAPUTO,Baixo,NaN


In [6]:
full_path = os.getcwd()
context = gx.get_context(context_root_dir = full_path.partition('notebooks')[0] + 'gx')

In [158]:
datasource_name = "project_data_cleaned"
try:
    datasource = context.sources.add_pandas(datasource_name)
except:
    print("Data Source already exists.")
    datasource = context.datasources[datasource_name]

In [159]:
data_asset_name = "project_ana"
try:
    data_asset = datasource.add_dataframe_asset(name=data_asset_name, dataframe= df_ana)
except:
    print("The data asset already exists. The required one will be loaded.")
    data_asset = datasource.get_asset(data_asset_name)

In [160]:
data_asset_name = "project_ref"
try:
    data_asset = datasource.add_dataframe_asset(name=data_asset_name, dataframe= df_ref)
except:
    print("The data asset already exists. The required one will be loaded.")
    data_asset = datasource.get_asset(data_asset_name)

## Build expectations

In [ ]:

from great_expectations.core import ExpectationSuite, ExpectationConfiguration

def build_expectation_suite(expectation_suite_name: str, feature_group: str) -> ExpectationSuite:
    """
    Builder used to retrieve an instance of the validation expectation suite.
    
    Args:
        expectation_suite_name (str): A dictionary with the feature group name and the respective version.
        feature_group (str): Feature group used to construct the expectations.
             
    Returns:
        ExpectationSuite: A dictionary containing all the expectations for this particular feature group.
    """
    
    expectation_suite = ExpectationSuite(
        expectation_suite_name=expectation_suite_name
    )
    
    #context = gx.get_context()
    #expectation_suite = context.add_expectation_suite("my_suite")
    #context.save_expectation_suite(expectation_suite)

    # target
    if feature_group == 'target':
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_values_to_be_of_type",
                        kwargs={"column": 'has_default', "type_": "int64"},
                    )
                )
        expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_in_set",
                    kwargs={
                        "column": "has_default",
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_not_be_null",
                kwargs={"column": 'has_default'}
            )
        )
    # numerical features
    if feature_group == 'numerical_features':
        
        integer_features = ['customer_id', 'duration_months', 'number_of_installments_to_pay',
       'run_date', 'previous_loan_count', 'previous_loan_defaults', 'active_loans_count', 'age', 'is_employed', 'is_married']
        float_features = ['credit_amount', 'avg_monthly_income', 'income_stability',
       'avg_monthly_expenses', 'expenses_stability', 'avg_monthly_funds',
       'funds_stability', 'previous_loans_avg_amount', 'previous_loans_std',
       'active_loan_amount_total', 'yr_net_monthly_in']
        
        # int
        for i in integer_features:
                expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_values_to_be_of_type",
                        kwargs={"column": i, "type_": "int64"},
                    )
                )
        for i in integer_features:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_between",
                    kwargs={
                        "column": i,
                        "min_value": -1,
                        "strict_min": False,
                        "max_value": None  # No upper bound
                    }
                )
            )
        # Id
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_not_be_null",
                kwargs={"column": "customer_id"}
            )
        )
        # float
        for i in float_features:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_of_type",
                    kwargs={"column": i, "type_": "float64"}
                )
            )
        for i in float_features:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_between",
                    kwargs={
                        "column": i,
                        "min_value": 0,
                        "strict_min": False,
                        "max_value": None  # No upper bound
                    }
                )
            )

        expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_between",
                    kwargs={
                        "column": 'age',
                        "min_value": 0,
                        "strict_min": True,
                        "max_value": 120  # No upper bound
                    }
                )
            )
        
        for i in ['is_employed', 'is_married']:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_in_set",
                    kwargs={
                        "column": i,
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite
                        "value_set": [0, 1]  # or ["yes", "no"]
                    }
                )
            )
        
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
       
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo', '0']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 
        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite

def get_validation_results(checkpoint_result):
    # validation_result is a dictionary containing one key-value pair
    validation_result_key, validation_result_data = next(iter(checkpoint_result["run_results"].items()))

    # Accessing the 'actions_results' from the validation_result_data
    validation_result_ = validation_result_data.get('validation_result', {})

    # Accessing the 'results' from the validation_result_data
    results = validation_result_["results"]
    meta = validation_result_["meta"]
    use_case = meta.get('expectation_suite_name')
    
    
    df_validation = pd.DataFrame({},columns=["Success","Expectation Type","Column","Column Pair","Max Value",\
                                       "Min Value","Element Count","Unexpected Count","Unexpected Percent","Value Set","Unexpected Value","Observed Value"])
    
    
    for result in results:
        # Process each result dictionary as needed
        success = result.get('success', '')
        expectation_type = result.get('expectation_config', {}).get('expectation_type', '')
        column = result.get('expectation_config', {}).get('kwargs', {}).get('column', '')
        column_A = result.get('expectation_config', {}).get('kwargs', {}).get('column_A', '')
        column_B = result.get('expectation_config', {}).get('kwargs', {}).get('column_B', '')
        value_set = result.get('expectation_config', {}).get('kwargs', {}).get('value_set', '')
        max_value = result.get('expectation_config', {}).get('kwargs', {}).get('max_value', '')
        min_value = result.get('expectation_config', {}).get('kwargs', {}).get('min_value', '')

        element_count = result.get('result', {}).get('element_count', '')
        unexpected_count = result.get('result', {}).get('unexpected_count', '')
        unexpected_percent = result.get('result', {}).get('unexpected_percent', '')
        observed_value = result.get('result', {}).get('observed_value', '')
        if type(observed_value) is list:
            #sometimes observed_vaue is not iterable
            unexpected_value = [item for item in observed_value if item not in value_set]
        else:
            unexpected_value=[]
        
        df_validation = pd.concat([df_validation, pd.DataFrame.from_dict( [{"Success" :success,"Expectation Type" :expectation_type,"Column" : column,"Column Pair" : (column_A,column_B),"Max Value" :max_value,\
                                           "Min Value" :min_value,"Element Count" :element_count,"Unexpected Count" :unexpected_count,"Unexpected Percent":unexpected_percent,\
                                                  "Value Set" : value_set,"Unexpected Value" :unexpected_value ,"Observed Value" :observed_value}])], ignore_index=True)
        
    return df_validation
# def build_expectation_suite(expectation_suite_name: str, feature_group: str) -> ExpectationSuite:
    """
    Builder used to retrieve an instance of the validation expectation suite.
    
    Args:
        expectation_suite_name (str): A dictionary with the feature group name and the respective version.
        feature_group (str): Feature group used to construct the expectations.
             
    Returns:
        ExpectationSuite: A dictionary containing all the expectations for this particular feature group.
    """
    
    expectation_suite = ExpectationSuite(
        expectation_suite_name=expectation_suite_name
    )
    

    # numerical features
    if feature_group == 'numerical_features':
        
        integer_features = ['no_of_dependents', 'segment_id', 'industry_id',
       'legal_doc_name1_id', 'customer_id', 'duration_months',
       'number_of_installments_to_pay', 'has_default', 'run_date',
       'previous_loan_count', 'previous_loan_defaults', 'active_loans_count']
        float_features = ['yr_net_monthly_in', 'credit_amount', 'avg_monthly_income',
       'income_stability', 'avg_monthly_expenses', 'expenses_stability',
       'avg_monthly_funds', 'funds_stability', 'previous_loans_avg_amount',
       'previous_loans_std', 'active_loan_amount_total']
        
        # int
        for i in integer_features:
                expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_values_to_be_of_type",
                        kwargs={"column": i, "type_": "int64"},
                    )
                )
        for i in integer_features:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_between",
                    kwargs={
                        "column": i,
                        "min_value": -1,
                        "strict_min": False,
                        "max_value": None  # No upper bound
                    }
                )
            )
        # Id
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_not_be_null",
                kwargs={"column": "customer_id"}
            )
        )
        # float
        for i in float_features:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_of_type",
                    kwargs={"column": i, "type_": "float64"}
                )
            )
        for i in float_features:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_between",
                    kwargs={
                        "column": i,
                        "min_value": 0,
                        "strict_min": False,
                        "max_value": None  # No upper bound
                    }
                )
            )
        
    # datetime features
    if feature_group == 'datetime_features':
        for i in ['customer_since', 'date_of_birth', 'birth_in_corp_date', 'legal_iss_date']:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_of_type",
                    kwargs={"column": i, 'type_':"datetime64[ns]"},
                )
            ) 
            # expectation_suite.add_expectation(
                # ExpectationConfiguration(
                #     expectation_type="expect_column_values_to_match_strftime_format",
                #     kwargs={"column": i, 'strftime_format': '%Y-%m-%d'}
                # )
            # )
        # CustomerSince
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_not_be_null",
                kwargs={"column": "customer_since"}
            )
        )
        # legal_exp_date
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_match_strftime_format",
                kwargs={"column": 'legal_exp_date', 'strftime_format': '%Y-%m-%d'}
            )
        )

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_pair_values_A_to_be_less_than_B",
                kwargs={
                    "column_A": "legal_iss_date	",
                    "column_B": "legal_exp_date"
                }
            )
        )
    
    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "customer_status", "value_set": ['Private Client - Standard', 'Corporate - Small', 'Corporate - Medium', 'Private Client High Networth', 'Corporate - Large', 'Financial - Large', 'Financial - Small', 'Proprietorship Standard', 'T24 Updates', 'Deceased Individual', 'Partnership firm Standard', 'Financial - Medium', 'Partnership High Networth', 'Customer Deletion', 'Governmental', 'Hotlisted', 'Proprietorship Highnetworth']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "employment_status", "value_set": ['MB', 'LP', 'OTHER', 'EMPLOYED', 'TPE', 'UNEMPLOYED', 'SELF-EMPLOYED', 'RET', 'UE', 'STUDENT', 'RETIRED']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "gender", "value_set": ['FEMALE', 'MALE']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "marital_status", "value_set": ['OTHER', 'DIVORCED', 'SINGLE', 'MARRIED', 'PARTNER', 'WIDOWED']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "cust_type", "value_set": ['RETAIL', 'CORPORATE', 'PROSPECT']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "nationality", "value_set": ['Portugal', 'Mozambique', 'India', 'Pakistan', 'Peoples Republic of China', 'Tokelau', 'South Africa', 'Turkey', 'Netherlands', 'Egypt', 'Switzerland', 'United States of America', 'Congo', 'Yemen', 'Uruguay', 'Afghanistan', 'Korea  Republic of', 'Australia', 'Morocco', 'Somalia', 'Spain', 'Bahrain', 'Lebanon', 'Kenya', 'Mauritius', 'Tanzania  United Republic of', 'Ethiopia', 'Guinea-Bissau', 'Mali', 'Mauritania', 'Guinea', 'Gambia', 'Sierra Leone', 'Mexico', 'Bangladesh', 'Aruba', 'United Arab Emirates', 'Zambia', 'Saudi Arabia', 'Singapore', 'Iran (Islamic Republic of)', 'Nigeria', 'Angola', 'Rwanda', 'Zimbabwe', 'Senegal', 'Hong Kong', 'New Zealand', 'Brazil', 'Italy', 'Cape Verde', 'Philippines', 'Malta', 'Oman', 'Jordan', 'Syrian Arab Republic', 'Libyan Arab Jamahiriya', 'Estado da Palestina', 'Malawi', 'Iraq', 'Great Britain', 'Turks and Caicos Islands', 'Germany', 'Japan', 'Namibia', 'Chile', 'Swaziland', 'Uzbekistan', 'France', 'Luxembourg', 'Norway', 'Bulgaria', 'Croatia', 'Canada', 'Ireland', 'Russian Federation', 'Burundi', 'Czech Republic', 'Uganda', 'Liberia', 'Poland', 'Belgium', 'Benin', 'Greece', 'Kyrgyzstan', 'American Samoa', 'Sao Tome and Principe', 'Tunisia', 'Haiti', 'Columbia', 'Macau', 'Romania', 'Cuba', 'Peru', 'Republic of China (Taiwan)', 'Kazakstan', 'Ghana', 'Finland', 'Puerto Rico', 'Vietnam', 'Israel', 'Thailand', 'Latvia', 'Sri Lanka', 'Nicaragua', 'Congo  Democratic Republic of the', 'Panama', 'Austria', 'Botswana', 'Indonesia', 'Madagascar', 'Comoro Islands', 'Cameroon', 'Nepal', 'Bhutan', 'Sudan', 'Sweden', 'Serbia', 'Mongolia', 'Paraguay', 'Argentina', 'Malaysia', 'Cyprus', 'Honduras', 'Guatemala', 'Monaco', 'Gabon', 'Denmark', 'Eritrea', 'Armenia', 'Ecuador', 'EU Countries', 'Venezuela', 'Dominica', 'Slovenia', 'Antigua And Barbuda', 'Lesotho', 'Georgia', 'Ivory Coast', 'Europa', 'Timor-leste', 'Cayman Islands', 'Northern Mariana Islands', 'San Marino', 'Dominican Republic', 'Chad', 'Algeria', 'Korea  Democratic Peoples Rep. of', 'Iceland', 'Turkmenistan', 'Ukraine', 'Bahamas', 'Bolivia', 'Niger', 'Liechtenstein', 'Hungary', 'Monserrat', 'Jersey', 'Estonia', 'Guadeloupe', 'Costa Rica', 'Lithuania', 'Guyana', 'Qatar', 'Andorra', 'Reunion', 'Yugoslavia', 'Belarus', 'Slovakia', 'Saint Lucia', 'Papua New Guinea', 'Barbados', 'Azerbaijan', 'Fiji', 'Holy See (Vatican City State)', 'Central African Republic']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "residence_code", "value_set": ['MZ', 'PT', 'ZA', 'NL', 'EG', 'CH', 'CN', 'US', 'IN', 'YE', 'UY', 'AF', 'KR', 'AU', 'ES', 'BH', 'LB', 'MU', 'TZ', 'KE', 'AW', 'AE', 'TR', 'AO', 'NG', 'ZM', 'RW', 'BR', 'ZW', 'MA', 'MR', 'IT', 'NZ', 'MT', 'OM', 'JP', 'CL', 'SZ', 'SA', 'FR', 'LU', 'HR', 'PL', 'GR', 'KG', 'ST', 'TN', 'IR', 'IE', 'GB', 'MW', 'VN', 'JO', 'FI', 'SE', 'PK', 'LV', 'BE', 'CV', 'CU', 'PA', 'MX', 'CO', 'DE', 'AT', 'BW', 'ID', 'MG', 'CA', 'IQ', 'HK', 'KM', 'CM', 'NO', 'CD', 'SG', 'RO', 'RS', 'GH', 'PY', 'PH', 'TW', 'LS', 'MY', 'BM', 'CZ', 'MC', 'AR', 'DK', 'PE', 'XE', 'SI', 'AG', 'TH', 'UG', 'RE', 'GE', 'GM', 'CI', 'KZ', 'CY', 'GA', 'GN', 'MO', 'KP', 'TC', 'IL', 'MN', 'BS', 'UA', 'RU', 'MP', 'HU', 'LR', 'EE', 'AM', 'CR', 'BD', 'SO', 'AD', 'SN', 'SK', 'LC', 'AS', 'BB', 'LT', 'BI', 'LK', 'ML', 'ET', 'EU', 'ER']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "residence_status", "value_set": ['HOME.OWNER', 'OTHER', 'TENANT', 'LIVING.WTH.PARENTS', 'SQUATTER']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "residence_type", "value_set": ['RESIDENTIAL.APT', 'INDEPEDENT.HOUSE', 'FARM.HOUSE', 'SERVICED.APT']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "title", "value_set": ['MRS', 'MR', 'DR', 'DRS', 'MISS', 'MAST1', 'ENG', 'PHD', 'MAST']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "cust_type1", "value_set": ['RETAIL', 'CORPORATE', 'PROSPECT']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "habliter", "value_set": ['Ate12 ano', 'Curso Superior', 'Bacharelato', 'Doutoramento', 'Mestrado', 'S.Estudos', 'Ensino Primario']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "province", "value_set": ['NAMPULA', 'CIDADE DE MAPUTO', 'MAPUTO', 'SOFALA', 'TETE', 'CABO DELGADO', 'NOT APPLICABLE', 'MANICA', 'ZAMBEZIA', 'NIASSA', 'INHAMBANE', 'GAZA']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "legal_doc_name1_id_description", "value_set": ['Licenca/Alvara', 'DIRE', 'Certidao da Conservatoria de Regist', 'Licenca / Alvara para o exercicio d', 'BI', 'Certidao de reserva de nome', 'Passaporte', 'Others (specify on free text field)', 'Certidao de Registo Comercial (Caso', 'Acta(s) da Assembleia Geral dos Acc', 'Escritura Publica da constituicao d', 'Certidao de Registo / Ministerio da', 'NUIT- Numero Unico de Identificacao', 'Cedula Pessoal', 'Recibo de pedido de BI', 'Cartao de identificacao de refugiad', 'NUIT-Numero Unico de Identificac', 'Carta de conducao', 'BR com o Pacto Social / Estatutos d', 'Termo de Autorizacao do Ministerio"', 'Certidao narrativa completa de nasc', 'Certidao da sentenca do Tribunal de', 'Numero de Registo Fiscal ( Modelo 4', 'Cartao de recenseamento eleitoral', 'Prova de Residencia', 'Prova documental de nao residente', 'Cartao de agricultor/trabalhador ab', 'Cartao de INSS', 'Documento de Transitario', 'Declaracao Comprovativa / Ministeri', 'Cedula militar', 'Documento de Regime Especial', 'Contrato de Trabalho', 'Documento de Embaixada', 'Procuracoes (especificar)', 'Documento de Staff Embaixada', 'Acta(s) da reuniao dos socios (pode', 'Documeto de Exportador']},
            )
        ) 
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo']},
            )
        ) 
        for i in ['placebrth', 'ocupation_desc', 'town_country', 'district', 'legal_iss_auth']:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_of_type",
                    kwargs={"column": i, "type_": "object"},
                )
            )
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "payment_frequency", "value_set": ['Monthly', 'Single', 'Quarterly']},
            )
        ) 

        expectation_suite.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_distinct_values_to_be_in_set",
                        kwargs={"column": "credit_type", "value_set": ['Personal Credit', 'Credit Card', 'Secured Current Account', 'Leasing', 'Public Sector Employee Loan', 'Arranged Overdraft', 'Business Loan Account']},
                    )
                ) 
    return expectation_suite

In [96]:
full_path = os.getcwd()
context = gx.get_context(context_root_dir = full_path.partition('notebooks')[0] + 'gx')

In [97]:
validation_expectation_suite_target = build_expectation_suite("target_expectations","target")
validation_expectation_suite_numerical = build_expectation_suite("numerical_expectations","numerical_features")
validation_expectation_suite_categorical = build_expectation_suite("categorical_expectations","categorical_features")
# validation_expectation_suite_datetime = build_expectation_suite("datetime_expectations","datetime_features")

context.add_or_update_expectation_suite(expectation_suite=validation_expectation_suite_target)
context.add_or_update_expectation_suite(expectation_suite=validation_expectation_suite_numerical)
context.add_or_update_expectation_suite(expectation_suite=validation_expectation_suite_categorical)
# context.add_or_update_expectation_suite(expectation_suite=validation_expectation_suite_datetime)

numerical_feature_descriptions =[]
categorical_feature_descriptions =[]
# datetime_feature_descriptions =[]

df_target = df[target].reset_index()
df_numeric = df[numerical_features].reset_index()
df_categorical = df[categorical_features].reset_index()
# df_datetime = df[datetime_features].reset_index()


In [98]:
# add datasource
datasource_name = "project_data_feature_engineered_test"
try:
    datasource = context.sources.add_pandas(datasource_name)
    print("Data Source created.")
except:
    print("Data Source already exists.")
    datasource = context.datasources[datasource_name]

def build_asset_and_checkpoint(asset_name, df, checkpoint_name, expectation_suite_name):
    data_asset_name = asset_name
    try:
        data_asset = datasource.add_dataframe_asset(name=data_asset_name, dataframe=df)
    except:
        data_asset = datasource.get_asset(data_asset_name)

    batch_request = data_asset.build_batch_request(dataframe=df)

    checkpoint = gx.checkpoint.SimpleCheckpoint(
        name=checkpoint_name,
        data_context=context,
        validations=[
            {
                "batch_request": batch_request,
                "expectation_suite_name": expectation_suite_name,
            },
        ],
    )
    return checkpoint

checkpoint_target = build_asset_and_checkpoint('target', df_target, 'checkpoint_target', 'target_expectations').run()
checkpoint_num = build_asset_and_checkpoint('numerical_features', df_numeric, 'checkpoint_num', 'numerical_expectations').run()
checkpoint_cat = build_asset_and_checkpoint('categorical_features', df_categorical, 'checkpoint_cat', 'categorical_expectations').run()
# checkpoint_date = build_asset_and_checkpoint('date_features', df_datetime, 'checkpoint_date', 'datetime_expectations').run()

df_val1 = get_validation_results(checkpoint_target)
df_val2 = get_validation_results(checkpoint_num)
df_val3 = get_validation_results(checkpoint_cat)
# df_val4 = get_validation_results(checkpoint_date)

df_validation = pd.concat([df_val1, df_val2, df_val3], ignore_index=True)

Data Source already exists.


Calculating Metrics:   0%|          | 0/13 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/163 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

In [99]:
df_validation[df_validation.Success == False]


,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value


In [85]:
df.columns

Index(['customer_id', 'credit_type', 'credit_amount', 'duration_months',
       'number_of_installments_to_pay', 'payment_frequency', 'has_default',
       'run_date', 'avg_monthly_income', 'income_stability',
       'avg_monthly_expenses', 'expenses_stability', 'avg_monthly_funds',
       'funds_stability', 'previous_loan_count', 'previous_loans_avg_amount',
       'previous_loans_std', 'previous_loan_defaults', 'active_loans_count',
       'active_loan_amount_total', 'seg_group', 'a_m_l_risk_rating',
       'yr_net_monthly_in', 'age', 'is_employed', 'is_married'],
      dtype='object')

In [48]:
df.columns

Index(['index', 'customer_since', 'customer_status', 'date_of_birth',
       'employment_status', 'gender', 'marital_status', 'placebrth',
       'cust_type', 'nationality', 'no_of_dependents', 'ocupation_desc',
       'residence_code', 'residence_status', 'residence_type', 'segment_id',
       'seg_group_x', 'title', 'town_country', 'birth_in_corp_date',
       'cust_type1', 'industry_id', 'habliter', 'province', 'district',
       'legal_doc_name1_id', 'legal_doc_name1_id_description',
       'legal_iss_date', 'legal_exp_date', 'legal_iss_auth',
       'a_m_l_risk_rating_x', 'yr_net_monthly_in_x', 'customer_id',
       'credit_type', 'credit_amount', 'duration_months',
       'number_of_installments_to_pay', 'payment_frequency', 'has_default',
       'run_date', 'avg_monthly_income', 'income_stability',
       'avg_monthly_expenses', 'expenses_stability', 'avg_monthly_funds',
       'funds_stability', 'previous_loan_count', 'previous_loans_avg_amount',
       'previous_loans_std', 'p

In [165]:
validation_expectation_suite_numerical = build_expectation_suite("numerical_expectations", "numerical_features")
validation_expectation_suite_datetime = build_expectation_suite("datetime_expectations", "datetime_features")

In [166]:
context.add_or_update_expectation_suite(expectation_suite=validation_expectation_suite_numerical)
context.add_or_update_expectation_suite(expectation_suite=validation_expectation_suite_datetime)

{
  "expectation_suite_name": "datetime_expectations",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_values_to_match_strftime_format",
      "kwargs": {
        "column": "customer_since",
        "strftime_format": "%Y-%m-%d"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_strftime_format",
      "kwargs": {
        "column": "date_of_birth",
        "strftime_format": "%Y-%m-%d"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_strftime_format",
      "kwargs": {
        "column": "birth_in_corp_date",
        "strftime_format": "%Y-%m-%d"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_strftime_format",
      "kwargs": {
        "column": "legal_iss_date",
        "strftime_format": "%Y-%m-%d"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_strftime_for

## Analysis

In [168]:
data_asset_analysis = datasource.get_asset("project_ana")

In [169]:
batch_request = data_asset_analysis.build_batch_request()

In [170]:
checkpoint = gx.checkpoint.SimpleCheckpoint(
    name="checkpoint_datetime",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": "datetime_expectations",
        },
    ],
)
checkpoint_result = checkpoint.run()

Calculating Metrics:   0%|          | 0/41 [00:00<?, ?it/s]

In [171]:
df_validation = get_validation_results(checkpoint_result)
df_validation

,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
0,True,expect_column_values_to_match_strftime_format,customer_since,"(, )",,,9761,0,0.0,,[],
1,True,expect_column_values_to_not_be_null,customer_since,"(, )",,,9761,0,0.0,,[],
2,True,expect_column_values_to_match_strftime_format,date_of_birth,"(, )",,,9761,0,0.0,,[],
3,True,expect_column_values_to_match_strftime_format,birth_in_corp_date,"(, )",,,9761,0,0.0,,[],
4,True,expect_column_values_to_match_strftime_format,legal_iss_date,"(, )",,,9761,0,0.0,,[],
5,True,expect_column_values_to_match_strftime_format,legal_exp_date,"(, )",,,9761,0,0.0,,[],


In [ ]:
df_validation[df_validation.Success == False]

In [133]:
checkpoint_num = gx.checkpoint.SimpleCheckpoint(
    name="checkpoint_numerical",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": "numerical_expectations",
        },
    ],
)
checkpoint_result = checkpoint_num.run()

Calculating Metrics:   0%|          | 0/48 [00:00<?, ?it/s]

In [134]:
df_validation = get_validation_results(checkpoint_result)
df_validation

,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
0,True,expect_column_values_to_be_of_type,no_of_dependents,"(, )",,,,,,,[],int64
1,True,expect_column_values_to_be_between,no_of_dependents,"(, )",None,-1,9761,0,0.0,,[],
2,True,expect_column_values_to_be_of_type,segment_id,"(, )",,,,,,,[],int64
3,True,expect_column_values_to_be_between,segment_id,"(, )",None,-1,9761,0,0.0,,[],
4,True,expect_column_values_to_be_of_type,industry_id,"(, )",,,,,,,[],int64
5,True,expect_column_values_to_be_between,industry_id,"(, )",None,-1,9761,0,0.0,,[],
6,True,expect_column_values_to_be_of_type,legal_doc_name1_id,"(, )",,,,,,,[],int64
7,True,expect_column_values_to_be_between,legal_doc_name1_id,"(, )",None,-1,9761,0,0.0,,[],
8,True,expect_column_values_to_be_of_type,new_id,"(, )",,,,,,,[],int64
9,True,expect_column_values_to_not_be_null,new_id,"(, )",,,9761,0,0.0,,[],


In [72]:
df_validation[df_validation.Success == False]


,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
